In [35]:
import pandas as pd
import numpy as np

In [36]:
df_adm = pd.read_csv('data/ADMISSIONS.csv')
df_adm.ADMITTIME = pd.to_datetime(df_adm.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DISCHTIME = pd.to_datetime(df_adm.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DEATHTIME = pd.to_datetime(df_adm.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])
df_adm = df_adm.reset_index(drop = True)
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
df_adm['NEXT_ADMISSION_TYPE'] = df_adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

rows = df_adm.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])

#When we filter out the "ELECTIVE", we need to correct the next admit time for these admissions since there might be 'emergency' next admit after "ELECTIVE"
df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
df_adm['DAYS_NEXT_ADMIT']=  (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)
df_adm['OUTPUT_LABEL'] = (df_adm.DAYS_NEXT_ADMIT < 30).astype('int')
### filter out newborn and death
df_adm = df_adm[df_adm['ADMISSION_TYPE']!='NEWBORN']
df_adm = df_adm[df_adm.DEATHTIME.isnull()]
df_adm['DURATION'] = (df_adm['DISCHTIME']-df_adm['ADMITTIME']).dt.total_seconds()/(24*60*60)

df_notes = pd.read_csv('data/NOTEEVENTS.csv')
df_notes = df_notes.sort_values(by=['SUBJECT_ID','HADM_ID','CHARTDATE'])
df_adm_notes = pd.merge(df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME','OUTPUT_LABEL','DURATION']],
                        df_notes[['SUBJECT_ID','HADM_ID','CHARTDATE','TEXT','CATEGORY']], 
                        on = ['SUBJECT_ID','HADM_ID'],
                        how = 'left')

df_adm_notes.ADMITTIME_C = df_adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])
df_adm_notes['ADMITTIME_C'] = pd.to_datetime(df_adm_notes.ADMITTIME_C, format = '%Y-%m-%d', errors = 'coerce')
df_adm_notes['CHARTDATE'] = pd.to_datetime(df_adm_notes.CHARTDATE, format = '%Y-%m-%d', errors = 'coerce')

### If Discharge Summary 
df_discharge = df_adm_notes[df_adm_notes['CATEGORY'] == 'Discharge summary']
# multiple discharge summary for one admission -> after examination -> replicated summary -> replace with the last one 
df_discharge = (df_discharge.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()
df_discharge=df_discharge[df_discharge['TEXT'].notnull()]

C:\Users\gaowt\AppData\Local\Temp\ipykernel_17864\3493349588.py:26: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_notes = pd.read_csv('data/NOTEEVENTS.csv')
C:\Users\gaowt\AppData\Local\Temp\ipykernel_17864\3493349588.py:33: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_adm_notes.ADMITTIME_C = df_adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])


In [37]:
import re

In [38]:
import string

In [39]:
from collections import defaultdict
from tqdm import tqdm

In [40]:
all_section_name = []

In [41]:
section_name_counter = defaultdict(int)

In [42]:
def get_sections(note):
    
    note = note.split('\n\n')
    # remove last several rows
    for i in range(len(note)-1, -1, -1):
        if note[i] == '':
#             print(note[i:])
            if len(note) - i <7:
                
                note = note[:i]
                
#             else:
#                 print(note[i:])
            break
            
    note = '\n\n'.join(note)
            
    note = note.splitlines()[1:]
#     print(note[0])
            
    # if there are new lines in between, and the following first row starts with a str with :, then consider it as 
    # the section name, otherwise, add the content to the previous section
    formated_content = []
    
    
    
    new_para = 1
    for row in note:
        row = row.strip()
#         print('row',row)
        if len(row) == 0:
            new_para = 1
            continue
        if new_para and row[-1] == ':':
            sec = row[:-1]
            section_name_counter[sec.lower()] += 1
            if sec.lower() not in all_section_name:
                all_section_name.append(sec.lower())
            formated_content.append([sec,''])
            new_para = 0
        else:
#             print(row[:27].lower())
            if ':' in row:
                idx = row.index(':')
                if row[:idx].lower() in all_section_name:
                    section_name_counter[row[:idx].lower()] += 1
                    lgth = len(row[:idx])
                    formated_content.append([row[:idx],row[lgth:]])
                    new_para = 0
                    continue
#             if row[:16].lower() == 'chief complaint:':
#                 formated_content.append(['chief complaint:',row[16:]])
#                 new_para = 0
#                 continue
#             if row[:27].lower() == 'history of present illness:':
#                 formated_content.append(['history of present illness:',row[27:]])
#                 new_para = 0
#                 continue
            if len(formated_content) == 0:
                formated_content.append(['basic information',row])
                section_name_counter['basic information'] += 1
            else:
#                 print('formated_content[-1][-1]',formated_content[-1][-1])
                formated_content[-1][-1] = formated_content[-1][-1]+' '+row
#     for sec,con in formated_content:
#         print('*'*20)
#         print('sec name:',sec)
#         print('content:',con)
        
#     print(all_section_name)
            
#         print(paragraph)
#         idx = paragraph.find("\n")
#         if idx != -1:
#         if ':' in paragraph:
#             colon_idx = paragraph.index(':')
#             section_name = paragraph[:colon_idx]
#             content = paragraph[colon_idx+1:]
#             formated_content.append([section_name,[content]])
#         else:
#             formated_content[-1][-1].append(paragraph)
#         else:
#             print('paragraph',paragraph)
#     print(formated_content)

In [45]:
tqdm.pandas()
df_discharge['TEXT'].progress_apply(get_sections)

100%|████████████████████████████████████████████████████████████████████████████| 43880/43880 [16:07<00:00, 45.34it/s]


0        None
1        None
2        None
3        None
4        None
         ... 
43875    None
43876    None
43877    None
43878    None
43879    None
Name: TEXT, Length: 43880, dtype: object

In [49]:
section_name_counter

{'service': 43800,
 'allergies': 40609,
 'past medical history': 40012,
 'history of present illness': 38905,
 'date of birth': 37854,
 'discharge medications': 36659,
 'social history': 36488,
 'discharge disposition': 34721,
 'medications on admission': 34204,
 'family history': 34047,
 'chief complaint': 34024,
 'discharge diagnosis': 33939,
 'discharge condition': 33486,
 'discharge instructions': 33304,
 'physical exam': 33053,
 'brief hospital course': 32285,
 'major surgical or invasive procedure': 32207,
 'followup instructions': 31705,
 'pertinent results': 31606,
 'impression': 30019,
 'heent': 25592,
 'facility': 24185,
 'neuro': 17767,
 'ext': 16740,
 'cv': 14740,
 'general': 14100,
 'provider': 13291,
 'abdomen': 13164,
 'mental status': 12221,
 'skin': 9518,
 'findings': 8690,
 'neck': 8494,
 'extremities': 7726,
 'vs': 7497,
 'lungs': 7045,
 'with': 6648,
 'hospital course': 6248,
 'cardiac': 5935,
 'vitals': 5865,
 'when': 5507,
 'physical examination': 4491,
 'pulses':

In [47]:
section_name_counter = {k:v for k,v in section_name_counter.items() if v >=10}

In [48]:
section_name_counter = dict(sorted(section_name_counter.items(), key=lambda item: item[1], reverse=True))

In [ ]:
section_name_counter

In [51]:
# selected_sections = list(section_name_counter.keys())[:23]
selected_sections = list(section_name_counter.keys())

In [52]:
def split_notes_to_sections(note):
    
    note = note.split('\n\n')
    # remove last several rows
    for i in range(len(note)-1, -1, -1):
        if note[i] == '':
#             print(note[i:])
            if len(note) - i <7:
                
                note = note[:i]
                
#             else:
#                 print(note[i:])
            break
            
    note = '\n\n'.join(note)
    note = note.strip()
            
    note = note.splitlines()[1:]
            
    # if there are new lines in between, and the following first row starts with a str with :, then consider it as 
    # the section name, otherwise, add the content to the previous section
    formated_content = []
    
    
    
    new_para = 1
    for row_i,row in enumerate(note):
        row = row.strip()
        if len(row.translate(str.maketrans('', '', string.punctuation))) == 0:
            continue

#         print('row_i',row_i,row)
        if len(row) == 0:
#             new_para = 1
            continue
            
#         print('row_i',row_i)
            
        if row_i <= 1:
            formated_content.append(['basic information',row])
#             section_name_counter['basic information'] += 1
            continue
#         if new_para and row[-1] == ':':
#             sec = row[:-1]
#             if sec.lower() in selected_sections:
#                 formated_content.append([sec,''])
#                 new_para = 0
#             else:
# #                 if len(formated_content) == 0:
# #                 print('row',row)
# #                 print('formated_content',len(formated_content),formated_content)
# #                 print('formated_content[-1]',formated_content[-1][-1])
                    
#                 formated_content[-1][-1] = formated_content[-1][-1]+' '+row            
#         else:
        pattern = re.compile("[0-9]+\.")

        if bool(pattern.match(row[:6])):
            re.sub(pattern, '', row[:6])
            new=re.sub(pattern,' ',row[:6]).strip()
            row = new + row[6:]
        if ':' in row:
            idx = row.index(':')
            if row[:idx].lower() in selected_sections:
#                     section_name_counter[row[:idx].lower()] += 1
                lgth = len(row[:idx])
                formated_content.append([row[:idx],row])
                new_para = 0

            else: 
#                     print('row',row_i,row)
                formated_content[-1][-1] = formated_content[-1][-1]+' '+row
#             continue
#             if len(formated_content) == 0:
#                 formated_content.append(['basic information',row])
#                 section_name_counter['basic information'] += 1
#             else:
#             print('row',row_i,row)
        else:
            formated_content[-1][-1] = formated_content[-1][-1]+' '+row
    return formated_content

In [53]:
df_discharge['sections'] = df_discharge['TEXT'].progress_apply(split_notes_to_sections)

100%|███████████████████████████████████████████████████████████████████████████| 43880/43880 [02:49<00:00, 258.63it/s]


In [54]:
df_discharge

SUBJECT_ID  HADM_ID           ADMITTIME           DISCHTIME  \
0               3   145834 2101-10-20 19:08:00 2101-10-31 13:58:00   
1               4   185777 2191-03-16 00:28:00 2191-03-23 18:41:00   
2               6   107064 2175-05-30 07:15:00 2175-06-15 16:00:00   
3              11   194540 2178-04-16 06:18:00 2178-05-11 19:00:00   
4              13   143045 2167-01-08 18:43:00 2167-01-15 15:15:00   
...           ...      ...                 ...                 ...   
43875       99985   176670 2181-01-27 02:47:00 2181-02-12 17:05:00   
43876       99991   151118 2184-12-24 08:30:00 2185-01-05 12:15:00   
43877       99992   197084 2144-07-25 18:03:00 2144-07-28 17:56:00   
43878       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
43879       99999   113369 2117-12-30 07:15:00 2118-01-04 16:30:00   

       DAYS_NEXT_ADMIT NEXT_ADMITTIME ADMISSION_TYPE DEATHTIME  OUTPUT_LABEL  \
0                  NaN            NaT      EMERGENCY       NaT             0   
1                  NaN            NaT      EMERGENCY       NaT             0   
2                  NaN            NaT       ELECTIVE       NaT             0   
3                  NaN            NaT      EMERGENCY       NaT             0   
4                  NaN            NaT      EMERGENCY       NaT             0   
...                ...            ...            ...       ...           ...   
43875              NaN            NaT      EMERGENCY       NaT             0   
43876              NaN            NaT       ELECTIVE       NaT             0   
43877              NaN            NaT      EMERGENCY       NaT             0   
43878              NaN            NaT       ELECTIVE       NaT             0   
43879              NaN            NaT       ELECTIVE       NaT             0   

        DURATION  CHARTDATE  \
0      10.784722 2101-10-31   
1       7.759028 2191-03-23   
2      16.364583 2175-06-15   
3      25.529167 2178-05-11   
4       6.855556 2167-01-15   
...          ...        ...   
43875  16.595833 2181-02-12   
43876  12.156250 2185-01-05   
43877   2.995139 2144-07-28   
43878   3.218750 2147-02-11   
43879   5.385417 2118-01-04   

                                                    TEXT           CATEGORY  \
0      Admission Date:  [**2101-10-20**]     Discharg...  Discharge summary   
1      Admission Date:  [**2191-3-16**]     Discharge...  Discharge summary   
2      Admission Date: [**2175-5-30**]        Dischar...  Discharge summary   
3      Admission Date:  [**2178-4-16**]              ...  Discharge summary   
4      Name:  [**Known lastname 9900**], [**Known fir...  Discharge summary   
...                                                  ...                ...   
43875  Admission Date:  [**2181-1-27**]              ...  Discharge summary   
43876  Admission Date:  [**2184-12-24**]             ...  Discharge summary   
43877  Admission Date:  [**2144-7-25**]              ...  Discharge summary   
43878  Admission Date:  [**2147-2-8**]              D...  Discharge summary   
43879  Admission Date:  [**2117-12-30**]             ...  Discharge summary   

      ADMITTIME_C                                           sections  
0      2101-10-20  [[basic information, Date of Birth:   [**2025-...  
1      2191-03-16  [[basic information, Date of Birth:   [**2143-...  
2      2175-05-30  [[basic information, Date of Birth:           ...  
3      2178-04-16  [[basic information, Date of Birth:  [**2128-2...  
4      2167-01-08  [[basic information, Admission Date:  [**2167-...  
...           ...                                                ...  
43875  2181-01-27  [[basic information, Date of Birth:  [**2127-4...  
43876  2184-12-24  [[basic information, Date of Birth:  [**2137-4...  
43877  2144-07-25  [[basic information, Date of Birth:  [**2078-1...  
43878  2147-02-08  [[Service, Service: SURGERY], [Allergies, Alle...  
43879  2117-12-30  [[basic information, Date of Birth:  [**2054-1...  

[43880 rows x 15 columns]

In [148]:
df_discharge.iloc[0]['sections']

[['basic information', 'Date of Birth:   [**2025-4-11**]     Sex:  M'],
 ['Service', 'Service:  Medicine'],
 ['CHIEF COMPLAINT',
  'CHIEF COMPLAINT:  Admitted from rehabilitation for hypotension (systolic blood pressure to the 70s) and decreased urine output.'],
 ['HISTORY OF PRESENT ILLNESS',
  'HISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old male who had been hospitalized at the [**Hospital1 190**] from [**10-11**] through [**10-19**] of [**2101**] after undergoing a left femoral-AT bypass graft and was subsequently discharged to a rehabilitation facility. On [**2101-10-20**], he presented again to the [**Hospital1 346**] after being found to have a systolic blood pressure in the 70s and no urine output for 17 hours. A Foley catheter placed at the rehabilitation facility yielded 100 cc of murky/brown urine.  There may also have been purulent discharge at the penile meatus at this time. On presentation to the Emergency Department, the patient was without subjective complaint

In [55]:
def preprocess1(x):
    # y=re.sub('\n [0-9]+\.','.',x) #remove 1.2. since the segmenter segments based on this
    
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    # y=re.sub('\n [0-9]+\.','.',y)
    # y=re.sub('(\n [0-9]+\.)','.',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('date of birth:','',y)
    y=re.sub('--|__|==','',y)

    y = y.replace('p.o.','')
    y = y.replace('q.d.','.')
    y = y.replace('b.i.d','.')
    y = y.replace('q.h.s.','.')
    y = y.replace(' q.','.')

    
    y = re.sub(r'\?+', "?", y)
    y=re.sub(' +', ' ', y)
    #y=re.sub(r'\.+', ". ", y)
#     y=re.sub(r'(\. )+', ". ", y)
    y=re.sub('\.\.+', '. ', y) 
    y=y.replace(':.', ':')
#     y=re.sub(r'\.+', ". ", y)

# #     y=re.sub(r'[. ]+','.',y)
#     y=y.replace('. ', '.')
#     y=re.sub(' +', ' ', y)
#     y = y.replace('..','.')
    
#     y=y.replace(':.', ':')
    y = y.strip()
    y = y.strip(string.punctuation)
    # y = y.strip('.')
    y = y.strip()
    y = y.replace(' . ','. ')
    y=re.sub(' +', ' ', y)
    
    return y

def remove_item(y):
    y=re.sub('\n [0-9]+\.',' ',y)
    y=re.sub('\n[0-9]+\.',' ',y)
    return y

def preprocessing(df_less_n): 
    
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n\n','. ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(remove_item)
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()
#     print(df_less_n)

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))
#     return df_less_n

    #to get 318 words chunks for readmission tasks
    #chunk_size = 318
    chunk_size = 290
    from tqdm import tqdm
    df_len = len(df_less_n)
    # want=pd.DataFrame({'ID':[],'TEXT':[],'Label':[]})
    ids = []
    texts = []
    labels = []
    for i in tqdm(range(df_len)):
        x=df_less_n.TEXT.iloc[i].split()
        n=int(len(x)/chunk_size)
        for j in range(n):
            ids.append(df_less_n.HADM_ID.iloc[i])
            texts.append(' '.join(x[j*chunk_size:(j+1)*chunk_size]).strip().strip(string.punctuation).strip())
            labels.append(df_less_n.OUTPUT_LABEL.iloc[i])
            # want=want.concat({'TEXT':' '.join(x[j*chunk_size:(j+1)*chunk_size]).strip().strip('.').strip(),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
        if len(x)%chunk_size>10:
            # want=want.concat({'TEXT':' '.join(x[-(len(x)%chunk_size):]).strip().strip('.').strip(),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
            ids.append(df_less_n.HADM_ID.iloc[i])
            texts.append(' '.join(x[-(len(x)%chunk_size):]).strip().strip(string.punctuation).strip())
            labels.append(df_less_n.OUTPUT_LABEL.iloc[i])
    want= pd.DataFrame({'ID':ids,'TEXT':texts,'Label':labels})
    
    return want


In [56]:
def preprocessing_sections_helper(section): 
    
#     section=section.fillna(' ')
    section=section.replace('\n\n','. ')
#     section=remove_item(section)
    section=section.replace('\n',' ')
    section=section.replace('\r',' ')
    section=section.strip()
    section=section.lower()
#     print(df_less_n)

    section=preprocess1(section)
    return section

def preprocessing_sections(row):
    out = []
#     print(row)
    for sec,content in row['sections']:
        out.append([sec,preprocessing_sections_helper(content)])
    return out
#     df['sections'] = out



In [57]:
df_discharge['processed_sections'] = df_discharge.apply(lambda row: preprocessing_sections(row), axis=1)

In [58]:
df_discharge.iloc[0]['processed_sections']

[['basic information', 'sex: m'],
 ['Service', 'service: medicine'],
 ['CHIEF COMPLAINT',
  'chief complaint: admitted from rehabilitation for hypotension (systolic blood pressure to the 70s) and decreased urine output'],
 ['HISTORY OF PRESENT ILLNESS',
  'history of present illness: the patient is a 76-year-old male who had been hospitalized at the from through of after undergoing a left femoral-at bypass graft and was subsequently discharged to a rehabilitation facility. on , he presented again to the after being found to have a systolic blood pressure in the 70s and no urine output for 17 hours. a foley catheter placed at the rehabilitation facility yielded 100 cc of murky/brown urine. there may also have been purulent discharge at the penile meatus at this time. on presentation to the emergency department, the patient was without subjective complaints. in the emergency department, he was found to have systolic blood pressure of 85. he was given 6 liters of intravenous fluids and tr

In [59]:
# ids = []
# texts = []
# labels = []
# for i in tqdm(range(df_len)):
#     x=df_less_n.TEXT.iloc[i].split()
#     n=int(len(x)/chunk_size)
#     for j in range(n):
#         ids.append(df_less_n.HADM_ID.iloc[i])
#         texts.append(' '.join(x[j*chunk_size:(j+1)*chunk_size]).strip().strip(string.punctuation).strip())
#         labels.append(df_less_n.OUTPUT_LABEL.iloc[i])
#         # want=want.concat({'TEXT':' '.join(x[j*chunk_size:(j+1)*chunk_size]).strip().strip('.').strip(),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
#     if len(x)%chunk_size>10:
#         # want=want.concat({'TEXT':' '.join(x[-(len(x)%chunk_size):]).strip().strip('.').strip(),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
#         ids.append(df_less_n.HADM_ID.iloc[i])
#         texts.append(' '.join(x[-(len(x)%chunk_size):]).strip().strip(string.punctuation).strip())
#         labels.append(df_less_n.OUTPUT_LABEL.iloc[i])
# want= pd.DataFrame({'ID':ids,'TEXT':texts,'Label':labels})

def split_sections(content,chunk_size):
    content = content.split('.')
    split_content = []
    text = ''
    for s in content:
        s = s.strip()
        if len(text + s) <= chunk_size:
            text = text + s
        else:
            split_content.append(text)
            text = s
    split_content.append(text)
    return split_content
    
def combine_sections(df):
    chunk_size = 290
    ids = []
    texts = []
    labels = []
    section_token_idx = []
    section_char_idx = []
    for index,row in tqdm(df.iterrows()):
        sections = row['processed_sections']
        char_idx = 0
        token_idx = 0
        sec_char_idx = []
        sec_token_idx = []
        combined_text = ''
        new_sections = []
#         print(sections)
        for sec,content in sections:
            if len(content.split()) > chunk_size:
                split_content = split_sections(content,chunk_size)
                for c in split_content:
                    new_sections.append([sec,c])
            else:
                new_sections.append([sec,content])
        for sec,content in sections:
            content = content.strip()
            if len((combined_text + ' ' + content).split()) > chunk_size:
                ids.append(row['HADM_ID'])
                texts.append(combined_text.strip())
                labels.append(row['OUTPUT_LABEL'])
                section_token_idx.append(sec_token_idx)
                section_char_idx.append(sec_char_idx)
                char_idx = 0
                token_idx = 0
                sec_char_idx = []
                sec_token_idx = []
                combined_text = ''
            combined_text = combined_text + ' ' + content
            sec_char_idx.append(char_idx)
            sec_token_idx.append(token_idx)
            char_idx = char_idx+len(content)+1
            token_idx = token_idx+len(content.split())
        ids.append(row['HADM_ID'])
        texts.append(combined_text)
        labels.append(row['OUTPUT_LABEL'])
        section_token_idx.append(sec_token_idx)
        section_char_idx.append(sec_char_idx)
    want= pd.DataFrame({'ID':ids,'TEXT':texts,'Label':labels,'Sec_tok_idx':section_token_idx,'Sec_char_idx':section_char_idx})
    return want
            
                
            
            
        

In [60]:
df_discharge

SUBJECT_ID  HADM_ID           ADMITTIME           DISCHTIME  \
0               3   145834 2101-10-20 19:08:00 2101-10-31 13:58:00   
1               4   185777 2191-03-16 00:28:00 2191-03-23 18:41:00   
2               6   107064 2175-05-30 07:15:00 2175-06-15 16:00:00   
3              11   194540 2178-04-16 06:18:00 2178-05-11 19:00:00   
4              13   143045 2167-01-08 18:43:00 2167-01-15 15:15:00   
...           ...      ...                 ...                 ...   
43875       99985   176670 2181-01-27 02:47:00 2181-02-12 17:05:00   
43876       99991   151118 2184-12-24 08:30:00 2185-01-05 12:15:00   
43877       99992   197084 2144-07-25 18:03:00 2144-07-28 17:56:00   
43878       99995   137810 2147-02-08 08:00:00 2147-02-11 13:15:00   
43879       99999   113369 2117-12-30 07:15:00 2118-01-04 16:30:00   

       DAYS_NEXT_ADMIT NEXT_ADMITTIME ADMISSION_TYPE DEATHTIME  OUTPUT_LABEL  \
0                  NaN            NaT      EMERGENCY       NaT             0   
1                  NaN            NaT      EMERGENCY       NaT             0   
2                  NaN            NaT       ELECTIVE       NaT             0   
3                  NaN            NaT      EMERGENCY       NaT             0   
4                  NaN            NaT      EMERGENCY       NaT             0   
...                ...            ...            ...       ...           ...   
43875              NaN            NaT      EMERGENCY       NaT             0   
43876              NaN            NaT       ELECTIVE       NaT             0   
43877              NaN            NaT      EMERGENCY       NaT             0   
43878              NaN            NaT       ELECTIVE       NaT             0   
43879              NaN            NaT       ELECTIVE       NaT             0   

        DURATION  CHARTDATE  \
0      10.784722 2101-10-31   
1       7.759028 2191-03-23   
2      16.364583 2175-06-15   
3      25.529167 2178-05-11   
4       6.855556 2167-01-15   
...          ...        ...   
43875  16.595833 2181-02-12   
43876  12.156250 2185-01-05   
43877   2.995139 2144-07-28   
43878   3.218750 2147-02-11   
43879   5.385417 2118-01-04   

                                                    TEXT           CATEGORY  \
0      Admission Date:  [**2101-10-20**]     Discharg...  Discharge summary   
1      Admission Date:  [**2191-3-16**]     Discharge...  Discharge summary   
2      Admission Date: [**2175-5-30**]        Dischar...  Discharge summary   
3      Admission Date:  [**2178-4-16**]              ...  Discharge summary   
4      Name:  [**Known lastname 9900**], [**Known fir...  Discharge summary   
...                                                  ...                ...   
43875  Admission Date:  [**2181-1-27**]              ...  Discharge summary   
43876  Admission Date:  [**2184-12-24**]             ...  Discharge summary   
43877  Admission Date:  [**2144-7-25**]              ...  Discharge summary   
43878  Admission Date:  [**2147-2-8**]              D...  Discharge summary   
43879  Admission Date:  [**2117-12-30**]             ...  Discharge summary   

      ADMITTIME_C                                           sections  \
0      2101-10-20  [[basic information, Date of Birth:   [**2025-...   
1      2191-03-16  [[basic information, Date of Birth:   [**2143-...   
2      2175-05-30  [[basic information, Date of Birth:           ...   
3      2178-04-16  [[basic information, Date of Birth:  [**2128-2...   
4      2167-01-08  [[basic information, Admission Date:  [**2167-...   
...           ...                                                ...   
43875  2181-01-27  [[basic information, Date of Birth:  [**2127-4...   
43876  2184-12-24  [[basic information, Date of Birth:  [**2137-4...   
43877  2144-07-25  [[basic information, Date of Birth:  [**2078-1...   
43878  2147-02-08  [[Service, Service: SURGERY], [Allergies, Alle...   
43879  2117-12-30  [[basic information, Date of Birth:  [**2054-1...   

                  

In [61]:
new_df=combine_sections(df_discharge)

43880it [00:43, 1014.98it/s]


In [62]:
new_df

ID                                               TEXT  Label  \
0       145834  sex: m service: medicine chief complaint: admi...      0   
1       145834  allergies: the patient has no known drug aller...      0   
2       145834  physical examination on presentation: initial ...      0   
3       145834  radiology/imaging: his admission chest x-ray d...      0   
4       145834  cardiovascular: the patient was initially admi...      0   
...        ...                                                ...    ...   
260757  137810   what activities you can and cannot do: ?\twhe...      0   
260758  113369  sex: f service: neurosurgery allergies: penici...      0   
260759  113369  brief hospital course: s/p plif l4-5 with doct...      0   
260760  113369  discharge medications: acetaminophen 325 mg ta...      0   
260761  113369   discharge disposition: extended care facility...      0   

                                              Sec_tok_idx  \
0                                      [0, 2, 4, 21, 156]   
1                                              [0, 8, 96]   
2                                                     [0]   
3                                                [0, 115]   
4                                                     [0]   
...                                                   ...   
260757                       [0, 125, 129, 134, 149, 154]   
260758  [0, 2, 4, 7, 11, 18, 50, 56, 59, 62, 132, 138,...   
260759                                           [0, 188]   
260760                                                [0]   
260761                    [0, 4, 6, 10, 12, 31, 186, 238]   

                                             Sec_char_idx  
0                                    [0, 7, 25, 151, 966]  
1                                            [0, 51, 577]  
2                                                     [0]  
3                                                [0, 823]  
4                                                     [0]  
...                                                   ...  
260757                       [0, 661, 706, 733, 840, 866]  
260758  [0, 7, 29, 62, 103, 151, 377, 419, 438, 457, 9...  
260759                                          [0, 1048]  
260760                                                [0]  
260761             [0, 37, 54, 100, 120, 270, 1161, 1467]  

[260762 rows x 5 columns]

In [63]:
df_discharge_subset = new_df

In [64]:
new_df.to_csv('note_split_by_sec.csv')

In [ ]:
df_discharge.iloc[0]['TEXT']

In [8]:
df_discharge_subset = preprocessing(df_discharge_subset)
# df_discharge_subset.iloc[0]['TEXT']

100%|██████████████████████████████████████████████████████████████████████████| 43880/43880 [00:13<00:00, 3290.09it/s]


In [230]:
df_discharge_subset

ID                                               TEXT  Label  \
0       145834  sex: m service: medicine chief complaint: admi...      0   
1       145834  allergies: the patient has no known drug aller...      0   
2       145834  physical examination on presentation: initial ...      0   
3       145834  radiology/imaging: his admission chest x-ray d...      0   
4       145834  cardiovascular: the patient was initially admi...      0   
...        ...                                                ...    ...   
260757  137810   what activities you can and cannot do: ?\twhe...      0   
260758  113369  sex: f service: neurosurgery allergies: penici...      0   
260759  113369  brief hospital course: s/p plif l4-5 with doct...      0   
260760  113369  discharge medications: acetaminophen 325 mg ta...      0   
260761  113369   discharge disposition: extended care facility...      0   

                                              Sec_tok_idx  \
0                                      [0, 2, 4, 21, 156]   
1                                              [0, 8, 96]   
2                                                     [0]   
3                                                [0, 115]   
4                                                     [0]   
...                                                   ...   
260757                       [0, 125, 129, 134, 149, 154]   
260758  [0, 2, 4, 7, 11, 18, 50, 56, 59, 62, 132, 138,...   
260759                                           [0, 188]   
260760                                                [0]   
260761                    [0, 4, 6, 10, 12, 31, 186, 238]   

                                             Sec_char_idx  
0                                    [0, 7, 25, 151, 966]  
1                                            [0, 51, 577]  
2                                                     [0]  
3                                                [0, 823]  
4                                                     [0]  
...                                                   ...  
260757                       [0, 661, 706, 733, 840, 866]  
260758  [0, 7, 29, 62, 103, 151, 377, 419, 438, 457, 9...  
260759                                          [0, 1048]  
260760                                                [0]  
260761             [0, 37, 54, 100, 120, 270, 1161, 1467]  

[260762 rows x 5 columns]

In [99]:
df_discharge_subset.iloc[0]['TEXT']

"sex: m. service: medicine. chief complaint: admitted from rehabilitation for hypotension (systolic blood pressure to the 70s) and decreased urine output.. history of present illness: the patient is a 76-year-old male who had been hospitalized at the from through of after undergoing a left femoral-at bypass graft and was subsequently discharged to a rehabilitation facility.. on , he presented again to the after being found to have a systolic blood pressure in the 70s and no urine output for 17 hours. a foley catheter placed at the rehabilitation facility yielded 100 cc of murky/brown urine. there may also have been purulent discharge at the penile meatus at this time.. on presentation to the emergency department, the patient was without subjective complaints. in the emergency department, he was found to have systolic blood pressure of 85. he was given 6 liters of intravenous fluids and transiently started on dopamine for a systolic blood pressure in the 80.s. past medical history: coro

In [39]:
def old_preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)

    
    
    y=re.sub('--|__|==','',y)
    y=re.sub(r'\.+', ". ", y)

    # y=re.sub(r'[. ]+','.',y)
    y=y.replace('. ', '.')
    y=re.sub(' +', ' ', y)
    # y=y.replace(': .', ':')
    return y

def old_preprocessing(df_less_n): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' . ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: old_preprocess1(x))

    #to get 318 words chunks for readmission tasks
    chunk_size = 318
    from tqdm import tqdm
    df_len = len(df_less_n)
    # want=pd.DataFrame({'ID':[],'TEXT':[],'Label':[]})
    ids = []
    texts = []
    labels = []
    for i in tqdm(range(df_len)):
        x=df_less_n.TEXT.iloc[i].split()
        n=int(len(x)/chunk_size)
        for j in range(n):
            ids.append(df_less_n.HADM_ID.iloc[i])
            texts.append(' '.join(x[j*chunk_size:(j+1)*chunk_size]).strip().strip(string.punctuation).strip())
            labels.append(df_less_n.OUTPUT_LABEL.iloc[i])
            # want=want.append({'TEXT':' '.join(x[j*chunk_size:(j+1)*chunk_size]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
        if len(x)%chunk_size>10:
            ids.append(df_less_n.HADM_ID.iloc[i])
            texts.append(' '.join(x[-(len(x)%chunk_size):]).strip().strip(string.punctuation).strip())
            labels.append(df_less_n.OUTPUT_LABEL.iloc[i])
            # want=want.append({'TEXT':' '.join(x[-(len(x)%chunk_size):]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
    want= pd.DataFrame({'ID':ids,'TEXT':texts,'Label':labels})
    return want

In [21]:
df_discharge_subset = df_discharge[:5000]

In [22]:
df_discharge_subset = old_preprocessing(df_discharge_subset)

/var/folders/21/cx6f7lnx12g3yfy7b3r1tr2h0000gn/T/ipykernel_1739/882964965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
/var/folders/21/cx6f7lnx12g3yfy7b3r1tr2h0000gn/T/ipykernel_1739/882964965.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' . ')
/var/folders/21/cx6f7lnx12g3yfy7b3r1tr2h0000gn/T/ipykernel_1739/882964965.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [65]:
import spacy

In [66]:
#nlp = spacy.load("en_core_sci_sm")
nlp = spacy.load("en_core_sci_md")

In [67]:
from tqdm import tqdm

In [ ]:
count = 0
for index,row in tqdm(df_discharge_subset.iterrows()):
    text = row['TEXT']
    #text = re.sub('.*\..*\..*', '.', text)
#     text = text.replace('. .','. ')
    for sent in text.split('.'):
        print(sent)
        if len(sent.split()) > 50:
            print(sent.split(','))
#     text = nlp(text)
#     for sent in text.sents:
#         if len(sent.text.split()) > 50:
#             # print(sent.text.split())
#             count += 1
# print(count)
#         print(len(sent.text.split()))

In [50]:
count = 0
for index,row in tqdm(df_discharge_subset.iterrows()):
    text = row['TEXT']
    #text = re.sub('.*\..*\..*', '.', text)
#     text = text.replace('. .','. ')
    text = nlp(text)
    for sent in text.sents:
        if len(sent.text.split()) > 50:
            # if 'pertinent results' not in sent.text:
            #     print(sent.text.split('.'))
            # else:
            #     print(sent.text)
            # print(sent.text.split('.'))
            count += 1
print(count)

21213it [18:11, 19.44it/s]

5989


In [ ]:
df_discharge_subset.iloc[0]['TEXT'].strip('.')

In [ ]:
text = df_discharge_subset.iloc[1]['TEXT']
text

In [68]:
import scispacy
import spacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.hyponym_detector import HyponymDetector
from scispacy.linking import EntityLinker
import pandas as pd
from collections import defaultdict
from collections import Counter

In [69]:
counter = Counter()
def map2_tokenized_pos(text,char_pos):
	pos1=len(text[:char_pos[0]+1].split())-1
	pos2=pos1+len(text[char_pos[0]:char_pos[1]].split())
	return [pos1,pos2]


In [70]:
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
linker = nlp.get_pipe("scispacy_linker")

C:\Users\gaowt\Anaconda3\envs\allennlp_env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\gaowt\Anaconda3\envs\allennlp_env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [71]:
tui_exclude = ['T040', 'T170', 'T080', 'T053', 'T032', 'T057', 'T073', 'T078', 'T058', 'T097', 'T079', 'T090', 'T077', 'T099', 'T092', 'T185', 'T098', 'T016', 'T083', 'T067', 'T015', 'T021', 'T056', 'T100', 'T064', 'T051', 'T087', 'T102', 'T075', 'T095', 'T094']

In [105]:
#data = df_discharge_subset[df_discharge_subset['ID']==161160]
data = df_discharge_subset
data

ID                                               TEXT  Label  \
0       145834  sex: m service: medicine chief complaint: admi...      0   
1       145834  allergies: the patient has no known drug aller...      0   
2       145834  physical examination on presentation: initial ...      0   
3       145834  radiology/imaging: his admission chest x-ray d...      0   
4       145834  cardiovascular: the patient was initially admi...      0   
...        ...                                                ...    ...   
260757  137810   what activities you can and cannot do: ?\twhe...      0   
260758  113369  sex: f service: neurosurgery allergies: penici...      0   
260759  113369  brief hospital course: s/p plif l4-5 with doct...      0   
260760  113369  discharge medications: acetaminophen 325 mg ta...      0   
260761  113369   discharge disposition: extended care facility...      0   

                                              Sec_tok_idx  \
0                                      [0, 2, 4, 21, 156]   
1                                              [0, 8, 96]   
2                                                     [0]   
3                                                [0, 115]   
4                                                     [0]   
...                                                   ...   
260757                       [0, 125, 129, 134, 149, 154]   
260758  [0, 2, 4, 7, 11, 18, 50, 56, 59, 62, 132, 138,...   
260759                                           [0, 188]   
260760                                                [0]   
260761                    [0, 4, 6, 10, 12, 31, 186, 238]   

                                             Sec_char_idx new_TEXT  
0                                    [0, 7, 25, 151, 966]     None  
1                                            [0, 51, 577]     None  
2                                                     [0]     None  
3                                                [0, 823]     None  
4                                                     [0]     None  
...                                                   ...      ...  
260757                       [0, 661, 706, 733, 840, 866]     None  
260758  [0, 7, 29, 62, 103, 151, 377, 419, 438, 457, 9...     None  
260759                                          [0, 1048]     None  
260760                                                [0]     None  
260761             [0, 37, 54, 100, 120, 270, 1161, 1467]     None  

[260762 rows x 6 columns]

In [73]:
#sample_data = df_discharge_subset

In [106]:
exclude_map = ['mr','day','sex','job','dry','v5','v6','v1','v2','md','son','ill','man','yo','bed']
data['new_TEXT'] = None
for index,row in tqdm(data.iterrows()):
    text = row['TEXT']
    text = re.sub(r'\.+', ".", text)
    #print('text',text)
    
    doc = nlp(text)
    new_text = ''

    Sec_tok_idx = row['Sec_tok_idx']
    Sec_char_idx = row['Sec_char_idx']

    #old_Sec_tok_idx = row['Sec_tok_idx'].copy()
    #old_Sec_char_idx = row['Sec_char_idx'].copy()
    
    for sent_i, sent in enumerate(doc.sents):
        """
        print(sent.start_char, sent.end_char)
        print(doc.text[sent.start_char:sent.end_char])
        print(sent.text)
        """
        standardized = []
        entities = sent.ents
        for ent in entities:
            if len(ent._.kb_ents)>0:
                umls_ent = ent._.kb_ents[0]

                # for umls_ent in ent._.kb_ents:
                e, start, end, tok_start, tok_end = ent.text, ent.start_char, ent.end_char, ent.start, ent.end
                cui_code, normed_name, aliases, tui_code, definition = linker.kb.cui_to_entity[umls_ent[0]]
                if len(e) <= 3:
                    if e not in exclude_map and e.lower() not in normed_name.lower().split():
                        #print('before',text[start:end])
                        #text[start:end] = normed_name.lower()
                        standardized.append((start, end,e,normed_name.lower()))
                        #print('after',text[start:end])
                    #print(e,normed_name)
        #print(standardized)
        
        if len(standardized) > 0:
            new_sent = ''
            pre_idx = sent.start_char
            for convert_i,(s,e,pre,new) in enumerate(standardized):
                """
                print('check',len(doc.text),s,e)
                print(pre,new,doc.text[pre_idx:s],s,e)
                print('short',doc.text[pre_idx:s])
                print('long',doc.text[pre_idx:e])
                print('old',doc.text[s:e])
                """
                if convert_i == 0:
                    new_sent += doc.text[pre_idx:s] + new
                else:
                    new_sent += ' '+doc.text[pre_idx:s] + new
                pre_idx = e

                updated_tok_i = len(doc.text[:e].split())
                updated_char_i = len(doc.text[:e])
                for _i,tok_i in enumerate(Sec_tok_idx):
                    if tok_i >= updated_tok_i:
                        #print('pre',pre,len(pre.split()),Sec_tok_idx[_i])
                        #print('pre',Sec_tok_idx[_i],doc.text.split()[Sec_tok_idx[_i-1]:tok_i])
                        Sec_tok_idx[_i] = Sec_tok_idx[_i] + len(new.split()) - len(pre.split())
                        #print('after',new,len(new.split()),Sec_tok_idx[_i])
                        #print('after',Sec_tok_idx[_i],(new_text+' '+new_sent).split()[Sec_tok_idx[_i-1]:Sec_tok_idx[_i]])
                for _i,char_i in enumerate(Sec_char_idx):
                    if char_i >= updated_char_i:
                        #print('pre',doc.text.split()[Sec_char_idx[_i-1]:char_i])
                        Sec_char_idx[_i] = Sec_char_idx[_i] + len(new) - len(pre)
                        #print('after',(new_text+' '+new_sent).split()[Sec_char_idx[_i-1]:Sec_char_idx[_i]])
                #print(old_Sec_tok_idx,Sec_tok_idx)
                #print(old_Sec_char_idx,Sec_char_idx)
                        
            new_sent += doc.text[pre_idx:sent.end_char]
            """
            print('*****')
            print('old sent',sent.text)
            print('new',new_sent)
            print('*****')
            """
            
        else:
            new_sent = sent.text
        if sent_i == 0:
            new_text += new_sent
        else:
            new_text += ' '+new_sent
        
        """
        print('.....')
        print('new_text',new_text)
        """
        
        #print(text == new_text)
    data.at[index,'new_TEXT'] = new_text
    """
    if new_text != text:
        #print(old_Sec_tok_idx,Sec_tok_idx)
        #print(old_Sec_char_idx,Sec_char_idx)
        for _i in range(1,len(old_Sec_tok_idx)):
            print('pre',doc.text.split()[old_Sec_tok_idx[_i-1]:old_Sec_tok_idx[_i]])
            print('after',new_text.split()[Sec_tok_idx[_i-1]:Sec_tok_idx[_i]])
    
        for _i in range(1,len(old_Sec_char_idx)):
            print('pre',doc.text[old_Sec_char_idx[_i-1]:old_Sec_char_idx[_i]])
            print('after',new_text[Sec_char_idx[_i-1]:Sec_char_idx[_i]])
    """
        
            
        
                

260762it [14:51:37,  4.87it/s]


In [107]:
data

ID                                               TEXT  Label  \
0       145834  sex: m service: medicine chief complaint: admi...      0   
1       145834  allergies: the patient has no known drug aller...      0   
2       145834  physical examination on presentation: initial ...      0   
3       145834  radiology/imaging: his admission chest x-ray d...      0   
4       145834  cardiovascular: the patient was initially admi...      0   
...        ...                                                ...    ...   
260757  137810   what activities you can and cannot do: ?\twhe...      0   
260758  113369  sex: f service: neurosurgery allergies: penici...      0   
260759  113369  brief hospital course: s/p plif l4-5 with doct...      0   
260760  113369  discharge medications: acetaminophen 325 mg ta...      0   
260761  113369   discharge disposition: extended care facility...      0   

                                              Sec_tok_idx  \
0                                      [0, 2, 4, 21, 156]   
1                                              [0, 8, 96]   
2                                                     [0]   
3                                                [0, 115]   
4                                                     [0]   
...                                                   ...   
260757                       [0, 125, 129, 134, 149, 154]   
260758  [0, 2, 4, 7, 11, 18, 50, 56, 59, 62, 132, 138,...   
260759                                           [0, 193]   
260760                                                [0]   
260761                    [0, 4, 6, 10, 12, 31, 186, 238]   

                                             Sec_char_idx  \
0                                    [0, 7, 25, 151, 966]   
1                                            [0, 51, 577]   
2                                                     [0]   
3                                                [0, 823]   
4                                                     [0]   
...                                                   ...   
260757                       [0, 661, 706, 733, 840, 866]   
260758  [0, 7, 29, 62, 103, 151, 377, 419, 438, 457, 9...   
260759                                          [0, 1099]   
260760                                                [0]   
260761             [0, 37, 54, 100, 120, 270, 1161, 1467]   

                                                 new_TEXT  
0       sex: m service: medicine chief complaint: admi...  
1       allergies: the patient has no known drug aller...  
2       physical examination on presentation: initial ...  
3       radiology/imaging: his admission chest x-ray d...  
4       cardiovascular: the patient was initially admi...  
...                                                   ...  
260757   what activities you can and cannot do: ?\twhe...  
260758  sex: f service: neurosurgery allergies: penici...  
260759  brief hospital course: s/p plif l4-5 with doct...  
260760  discharge medications: acetaminophen 325 mg ta...  
260761   discharge disposition: extended care facility...  

[260762 rows x 6 columns]

In [108]:
data.to_csv('../rationale-robustness-main/data/mt_1212/temp.csv')

In [23]:
data = pd.read_csv('../rationale-robustness-main/data/mt_1128/temp.csv')

In [80]:
data['new_TEXT'] = data['new_TEXT'].apply(lambda x:x.strip(' '))

In [21]:
data['new_TEXT'] = data['new_TEXT'].apply(lambda x:x.replace('.','. '))


In [120]:
"""
record = defaultdict(list)
data['MT'] = None
data['dependency'] = None
# data['sentence_index'] = None
data['sentence_attribute'] = None
# data['mt_indices'] = None
removed_mt = defaultdict(list)
count = 0
"""


for index,row in tqdm(data[90071:].iterrows()):
    text = row['new_TEXT']
    text = re.sub(r'\.+', ".", text).strip()
    doc = nlp(text)
    extracted_mt = []
    extracted_dep = []
    sentence_idx = []
    sentence_attribute = []
    mt_indices = []
    flag = 0
    for sent_i, sent in enumerate(doc.sents):
        sentence_attr = [sent_i, sent.start, sent.text, [], None]
        entities = sent.ents
        mt_index_list = []
        for ent in entities:
            if len(ent._.kb_ents)>0:
                umls_ent = ent._.kb_ents[0]

                # for umls_ent in ent._.kb_ents:
                e, start, end, tok_start, tok_end = ent.text, ent.start_char, ent.end_char, ent.start, ent.end
                cui_code, normed_name, aliases, tui_code, definition = linker.kb.cui_to_entity[umls_ent[0]]
                
                tui_code = [x for x in tui_code if x not in tui_exclude]
                
                if len(tui_code)>0:
                    flag = 1
                    extracted_mt.append((e, normed_name, [tok_start, tok_end], [start, end], cui_code, tui_code))
                    sentence_attr[3].append((e, normed_name, [tok_start, tok_end], [start, end], cui_code, tui_code))
                    mt_index_list.extend(list(range(tok_start, tok_end)))
                    
        dependency = defaultdict(list)
        for token in sent:
            # print(sent.start_char, token.idx, len(token.text))
            t_start = token.i-sent.start
            t_end = t_start + 1
            
            extracted_dep.append((token.dep_, token.text, token.head.text, t_start, token.head.i-sent.start, token.pos_))
            dependency[t_start].append((token.dep_, token.text, token.head.text, t_start, token.head.i-sent.start, token.pos_, token.head.pos_))
            
        sentence_attr[4] = dict(dependency)
        sentence_attr.append(mt_index_list)
        # mt_indices.append(mt_index_list)
        sentence_attribute.append(sentence_attr)
        # if flag:
        # 	sentence_idx.append(sent.start)
        # print(sentence_attr)
    # break
    data.at[index, 'MT'] = extracted_mt
    data.at[index, 'dependency'] = extracted_dep
    # data.at[index, 'sentence_index'] = sentence_idx
    data.at[index, 'sentence_attribute'] = sentence_attribute
    # data.at[index, 'mt_indices'] = mt_indices

    count += 1
    if count %50000 == 0:

        data.to_csv('../rationale-robustness-main/data/mt_1128/temp_1.csv')
#data = data[['ID','new_TEXT','Label','MT','dependency','sentence_attribute']]

170691it [8:45:41,  5.41it/s] 


In [121]:
data

ID                                               TEXT  Label  \
0       145834  sex: m service: medicine chief complaint: admi...      0   
1       145834  allergies: the patient has no known drug aller...      0   
2       145834  physical examination on presentation: initial ...      0   
3       145834  radiology/imaging: his admission chest x-ray d...      0   
4       145834  cardiovascular: the patient was initially admi...      0   
...        ...                                                ...    ...   
260757  137810   what activities you can and cannot do: ?\twhe...      0   
260758  113369  sex: f service: neurosurgery allergies: penici...      0   
260759  113369  brief hospital course: s/p plif l4-5 with doct...      0   
260760  113369  discharge medications: acetaminophen 325 mg ta...      0   
260761  113369   discharge disposition: extended care facility...      0   

                                              Sec_tok_idx  \
0                                      [0, 2, 4, 21, 156]   
1                                              [0, 8, 96]   
2                                                     [0]   
3                                                [0, 115]   
4                                                     [0]   
...                                                   ...   
260757                       [0, 125, 129, 134, 149, 154]   
260758  [0, 2, 4, 7, 11, 18, 50, 56, 59, 62, 132, 138,...   
260759                                           [0, 193]   
260760                                                [0]   
260761                    [0, 4, 6, 10, 12, 31, 186, 238]   

                                             Sec_char_idx  \
0                                    [0, 7, 25, 151, 966]   
1                                            [0, 51, 577]   
2                                                     [0]   
3                                                [0, 823]   
4                                                     [0]   
...                                                   ...   
260757                       [0, 661, 706, 733, 840, 866]   
260758  [0, 7, 29, 62, 103, 151, 377, 419, 438, 457, 9...   
260759                                          [0, 1099]   
260760                                                [0]   
260761             [0, 37, 54, 100, 120, 270, 1161, 1467]   

                                                 new_TEXT  \
0       sex: m service: medicine chief complaint: admi...   
1       allergies: the patient has no known drug aller...   
2       physical examination on presentation: initial ...   
3       radiology/imaging: his admission chest x-ray d...   
4       cardiovascular: the patient was initially admi...   
...                                                   ...   
260757   what activities you can and cannot do: ?\twhe...   
260758  sex: f service: neurosurgery allergies: penici...   
260759  brief hospital course: s/p plif l4-5 with doct...   
260760  discharge medications: acetaminophen 325 mg ta...   
260761   discharge disposition: extended care facility...   

                                                       MT  \
0       [(admitted from, Admitted from, [9, 11], [42, ...   
1       [(allergies, Hypersensitivity, [0, 1], [0, 9],...   
2       [(heart rate, heart rate, [15, 17], [117, 127]...   
3       [(radiology/imaging, Radiology/imaging, [0, 1]...   
4       [(cardiovascular, Cardiovascular system, [0, 1...   
...                                                   ...   
260757  [(activities, Activities, [1, 2], [5, 15], C04...   
260758  [(neurosurgery allergies, Science of neurosurg...   
260759  [(surgery, Surgery specialty, [12, 13], [50, 5...   
260760  [(discharge medications, Medication.discharge,...   
260761  [(discharge disposition, Discharge disposition...   

                                               dependency  \
0       [(compound, sex, service, 0, 3, NOUN), (punct,...   
1       [(nsubj, allergies, has, 0, 4, NOUN),

In [122]:
data = data[['ID','new_TEXT','Label','MT','dependency','sentence_attribute','Sec_tok_idx','Sec_char_idx']]

In [123]:
data.to_csv('../rationale-robustness-main/data/mt_1212/temp_1.csv')

In [ ]:
'mr','day','sex','job','dry','v5','v6','md','son','ill','man','yo','bed'

In [ ]:
data

In [124]:
old_train = pd.read_csv('../rationale-robustness-main/data/mt_new_new/train.csv')
old_test = pd.read_csv('../rationale-robustness-main/data/mt_new_new/test.csv')
old_dev = pd.read_csv('../rationale-robustness-main/data/mt_new_new/dev.csv')


In [125]:
old_train_id = set(old_train['ID'].values.tolist())
old_test_id = set(old_test['ID'].values.tolist())
old_dev_id = set(old_dev['ID'].values.tolist())


In [126]:
new_train = data[data['ID'].isin(old_train_id)]
new_train

ID                                           new_TEXT  Label  \
44      109451  service: medicine allergies: patient recorded ...      0   
45      109451  history of present illness: 87 yo m with chron...      0   
46      109451  past medical history: esrd secondary to hypert...      0   
47      109451  comments: right heart catheterization revealed...      0   
48      109451  final diagnosis: three vessel coronary artery ...      0   
...        ...                                                ...    ...   
260667  157197  cta chest: 1. no pulmonary embolism. bibasilar...      0   
260668  157197  brief hospital course: 49 y.o male with pmhx o...      0   
260669  157197  medications on admission: nacl nasal spray chl...      0   
260670  157197  discharge medications: chlorhexidine gluconate...      0   
260671  157197   discharge instructions: dear mr , you were ad...      0   

                                                       MT  \
44      [(medicine allergies, Drug Allergy, [2, 4], [9...   
45      [(illness, Illness (finding), [3, 4], [19, 26]...   
46      [(medical history, Medical History, [1, 3], [5...   
47      [(right heart catheterization, Catheterization...   
48      [(diagnosis, Diagnosis, [1, 2], [6, 15], C0011...   
...                                                   ...   
260667  [(cta chest, Chest, [0, 2], [0, 9], C0817096, ...   
260668  [(ivc filter, Inspiratory Vital Capacity Test,...   
260669  [(medications, Pharmaceutical Preparations, [0...   
260670  [(discharge medications, Medication.discharge,...   
260671  [(discharge instructions, Discharge instructio...   

                                               dependency  \
44      [(ROOT, service, service, 0, 0, NOUN), (punct,...   
45      [(nsubj, history, s/p, 0, 12, NOUN), (case, of...   
46      [(amod, past, history, 0, 2, ADJ), (amod, medi...   
47      [(nsubj, comments, revealed, 0, 5, NOUN), (pun...   
48      [(amod, final, diagnosis, 0, 1, ADJ), (ROOT, d...   
...                                                   ...   
260667  [(ROOT, cta, cta, 0, 0, VERB), (dobj, chest, c...   
260668  [(amod, brief, course, 0, 2, ADJ), (compound, ...   
260669  [(nsubj, medications, lotrimin, 0, 24, NOUN), ...   
260670  [(compound, discharge, 2, 0, 161, VERB), (dobj...   
260671  [(advcl, discharge, admitted, 0, 8, VERB), (do...   

                                       sentence_attribute  \
44      [[0, 0, service: medicine allergies: patient r...   
45      [[0, 0, history of present illness: 87 yo m wi...   
46      [[0, 0, past medical history: esrd secondary t...   
47      [[0, 0, comments: right heart catheterization ...   
48      [[0, 0, final diagnosis: three vessel coronary...   
...                                                   ...   
260667  [[0, 0, cta chest: 1. no pulmonary embolism., ...   
260668  [[0, 0, brief hospital course: 49 y.o male wit...   
260669  [[0, 0, medications on admission: nacl nasal s...   
260670  [[0, 0, discharge medications: chlorhexidine g...   
260671  [[0, 0, discharge instructions: dear mr , you ...   

                                              Sec_tok_idx  \
44                                     [0, 2, 13, 18, 35]   
45                                               [0, 266]   
46      [0, 72, 74, 92, 97, 99, 101, 114, 123, 131, 14...   
47                                                    [0]   
48                                            [0, 73, 76]   
...                                                   ...   
260667                                                [0]   
260668                                                [0]   
260669                                                [0]   
260670                       [0, 196, 200, 201, 209, 211]   
260671             [0, 117, 125, 130, 148, 153, 171, 176]   

                                             Sec_char_idx  
44                                  [0, 18, 94, 131, 258]  
45                                             

In [127]:
new_test = data[data['ID'].isin(old_test_id)]
new_dev = data[data['ID'].isin(old_dev_id)]


In [172]:
data.iloc[0]['new_TEXT']

" sex: m. service: medicine. chief complaint: admitted from rehabilitation for hypotension (systolic blood pressure to the 70s) and decreased urine output. history of present illness: the patient is a 76-year-old male who had been hospitalized at the from through of after undergoing a left femoral-at bypass graft and was subsequently discharged to a rehabilitation facility. on , he presented again to the after being found to have a systolic blood pressure in the 70s and no urine output for 17 hours. a foley catheter placed at the rehabilitation facility yielded 100 cc of murky/brown urine. there may also have been purulent discharge at the penile meatus at this time. on presentation to the emergency department, the patient was without subjective complaints. in the emergency department, he was found to have systolic blood pressure of 85. he was given 6 liters of intravenous fluids and transiently started on dopamine for a systolic blood pressure in the 80.s. past medical history: corona

In [130]:
new_dev

ID                                           new_TEXT  Label  \
424     126055  sex: f service: medicine allergies: penicillin...      1   
425     126055  past medical history: - sle dx ( - 16 years ol...      1   
426     126055  pertinent results: 08:50am blood wbc-3.5* rbc-...      1   
427     126055  blood cultures: all negative electrocardiogram...      1   
428     126055  ct of the abdomen without iv contrast: there h...      1   
...        ...                                                ...    ...   
260243  154665  hypotension: dichlorodiphenyl dichloroethylene...      1   
260244  154665  depression: home abilify given concern for cau...      1   
260245  154665  discharge medications: rifaximin 550 mg tablet...      1   
260246  154665  discharge disposition: extended care facility:...      1   
260247  154665   the following medications were changed during...      1   

                                                       MT  \
424     [(medicine allergies, Drug Allergy, [5, 7], [1...   
425     [(medical history, Medical History, [1, 3], [5...   
426     [(primed lymphocyte test, Primed lymphocyte te...   
427     [(blood cultures, Blood culture, [0, 2], [0, 1...   
428     [(abdomen, Abdomen, [3, 4], [10, 17], C0000726...   
...                                                   ...   
260243  [(hypotension, Hypotension, [0, 1], [0, 11], C...   
260244  [(depression, Mental Depression, [0, 1], [0, 1...   
260245  [(discharge medications, Medication.discharge,...   
260246  [(discharge disposition, Discharge disposition...   
260247  [(medications, Pharmaceutical Preparations, [2...   

                                               dependency  \
424     [(nsubj, sex, tunneled, 0, 37, NOUN), (punct, ...   
425     [(amod, past, history, 0, 2, ADJ), (amod, medi...   
426     [(amod, pertinent, results, 0, 1, ADJ), (nsubj...   
427     [(compound, blood, cultures, 0, 1, NOUN), (ROO...   
428     [(ROOT, ct, ct, 0, 0, NOUN), (case, of, abdome...   
...                                                   ...   
260243  [(ROOT, hypotension, hypotension, 0, 0, NOUN),...   
260244  [(nsubj, depression, recommend, 0, 25, NOUN), ...   
260245  [(ROOT, discharge, discharge, 0, 0, VERB), (do...   
260246  [(compound, discharge, disposition, 0, 1, VERB...   
260247  [(det, the, medications, 0, 2, DET), (amod, fo...   

                                       sentence_attribute  \
424     [[0, 0, sex: f service: medicine allergies: pe...   
425     [[0, 0, past medical history: - sle dx ( - 16 ...   
426     [[0, 0, pertinent results: 08:50am blood wbc-3...   
427     [[0, 0, blood cultures: all negative electroca...   
428     [[0, 0, ct of the abdomen without iv contrast:...   
...                                                   ...   
260243  [[0, 0, hypotension: dichlorodiphenyl dichloro...   
260244  [[0, 0, depression: home abilify given concern...   
260245  [[0, 0, discharge medications: rifaximin 550 m...   
260246  [[0, 0, discharge disposition: extended care f...   
260247  [[0, 0, the following medications were changed...   

                                             Sec_tok_idx  \
424                                 [0, 2, 4, 9, 18, 36]   
425     [0, 165, 200, 223, 226, 237, 240, 246, 248, 265]   
426                                                  [0]   
427                                                  [0]   
428                                        [0, 239, 263]   
...                                                  ...   
260243                                          [0, 144]   
260244                              [0, 32, 89, 91, 159]   
260245                                               [0]   
260246                      [0, 4, 6, 8, 21, 31, 33, 55]   
260247  [0, 168, 179, 184, 195, 200, 215, 220, 239, 244]   

                                             Sec_char_idx  
424                              [0, 7, 25, 69, 149, 291]  
425     [0, 1118, 1327, 1536, 1550, 1612, 1632, 1663, ...  

In [131]:
new_train.to_csv('../rationale-robustness-main/data/mt_1212/train.csv')
new_test.to_csv('../rationale-robustness-main/data/mt_1212/test.csv')
new_dev.to_csv('../rationale-robustness-main/data/mt_1212/dev.csv')

In [9]:
test = pd.read_csv('../rationale-robustness-main/data/mt_1103/test.csv')